In [ ]:
%load_ext autoreload
%autoreload 2
#%debug
%run -i run_genfeats.py --sources_type parcel_aal --bands crude\
 --src_grouping_fn 10\
 --src_grouping 0 \
 --raw S99_off_move \
 --feat_types con,H_act,H_mob,H_compl\
 --Kalman_smooth 0\
 --load_TFR 0\
 --load_CSD 0\
 --save_TFR 0\
 --save_CSD 0\
 --use_existing_TFR 0\
--stats_fn_prefix stats_S97,S99_3_
# --show_plots 1\
# --plot_types feat_timecourse
#--plot_types raw_timecourse
#--plot_types raw_stats,raw_timecourse,raw_psd,feat_stats,feat_timecourse
 #--plot_only 0

In [ ]:
assert chnames_tfr[0][0:3] == 'LFP'
assert np.all( [chnames_tfr[i] == subfeature_order[i] for i in range(len(chnames_tfr)) ] )

In [ ]:
ind_distr, ind_distr_parcels, ind_pairs_parcelsLFP, \
    parcel_couplings, LFP2parcel_couplings, LFP2LFP_couplings = res_couplings


In [ ]:
for chn_tpl,chi_tpl_list in LFP2LFP_couplings.items():
    print(chn_tpl,chi_tpl_list)
    for chi_tpl in chi_tpl_list:
        print( chnames_tfr[chi_tpl[0]], chnames_tfr[chi_tpl[1]] )

In [ ]:
chnames_nicened

In [ ]:
for pc in LFP2parcel_couplings:
    for (i,j) in LFP2parcel_couplings[pc]:
        print(pc, chnames_tfr[i],chnames_tfr[j], chnames_nicened[j] )

In [ ]:
_,names_src = utsne.selFeatsRegex(None, subfeature_order, ['msrc.*'])
_,names_lfp = utsne.selFeatsRegex(None, subfeature_order, ['LFP.*'])
chnames_tfr = list(names_src) + list(names_lfp)

In [ ]:
log_during_csd, log_before_bandaver, normalize_TFR 

In [ ]:
log_during_csd, log_before_bandaver, normalize_TFR = False,False,True

In [ ]:
tfrres_cur = tfrres_pri[0]

tfrres_cur_ = tfrres_cur
if normalize_TFR:
    tfrres_cur_ =  (tfrres_cur - tfr_mean[:,:,None] ) / tfr_std[:,:,None]

csd_cur, csdord = utils.tfr2csd(tfrres_cur_, sfreq, returnOrder=1,
                                            ind_pairs=None,
                                            parcel_couplings=parcel_couplings,
                                            LFP2LFP_couplings=LFP2LFP_couplings,
                                            LFP2parcel_couplings=LFP2parcel_couplings,
                                            oldchns=chnames_tfr,
                                            newchns=newchns,
                                            res_group_id=newchn_grouping_ind,
                                            log=log_during_csd)


bpow_abscsd_pri, bpow_imagcsd, csdord_strs_pri, csdord_strs_HFO_pri,bpow_abscsd_LFP_HFO_pri  = \
    utils.bandAverage( freqs,freqs_inc_HFO,csd_pri,csdord_pri,csdord_LFP_HFO_pri,
           csd_LFP_HFO_pri, fbands,fband_names, fband_names_inc_HFO,
           newchns, subfeature_order_lfp_hires, log_before_bandaver= log_before_bandaver )


In [ ]:
%matplotlib qt

abscsd_all = bpow_abscsd_pri[0]
s1,s2,s3 = abscsd_all.shape
abscsd_all= abscsd_all.reshape((s1*s2,s3))

In [ ]:
chnames_tfr

In [ ]:
supp_info_fname_noext = 'S99_off_move' + '_supp_info'
supp_info_fname = supp_info_fname_noext + '.npz'
supp_info_fname_full = os.path.join(gv.data_dir,supp_info_fname)
f = np.load(supp_info_fname_full,allow_pickle=1)
supp_info = f['supp_info'][()]

In [ ]:
csdord_ = csdord_pri[0]
anns = utils.intervals2anns( [ (a,b,'trem') for a,b in supp_info['trem_intervals'] ] )

abscsd = csd_pri[0]
freqis = np.where( (freqs >= 37) * (freqs <= 43) )[0]
freqis = np.where( (freqs >= 39) * (freqs <= 41) )[0]

abscsd = bpow_abscsd_pri[0]
#freqis = [0,1,2]
freqis = [1,2]
chis_sl = slice(-3,None,None)
chis_sl = slice(50,None,None)
chis_sl = slice(None,None,None)
#chis = np.where( csdord_[0] == csdord_[1] )[0]
chis = np.arange(abscsd.shape[0])
#abscsd = abscsd[chis_sl,freqis,:] 
abscsd = abscsd[chis,:,:][:,freqis,:]
s1,s2,s3 = abscsd.shape
abscsd= abscsd.reshape((s1*s2,s3))
#csdord_str_cur = csdord_strs[chis_sl]
csdord_str_cur = list( np.array(chnames_tfr)[chis] ) * len(freqis)
print(csdord_str_cur )

# regexs = ['.*{}.*'.format(chn) for chn in newchns[chis_sl] ]
# abscsd,csdord_str_cur = utsne.selFeatsRegex(abscsd,csdord_strs,regexs)
# print( len(csdord_str_cur) )

# regexs = ['{}_.*LFP.*LFP.*'.format('gamma')  ]
# regexs = ['.*LFP.*LFP.*'.format('gamma')  ]
# abscsd,csdord_str_cur = utsne.selFeatsRegex(abscsd_all,csdord_strs,regexs)
# print( len(csdord_str_cur) )


#abscsd = abscsd[:3:,:,:]  # some src where in raw data changes are seen
#if we take 3 channels for test dataset after reshape 46 and 47 show intersting stuff

if csdord_str_cur is not None:
    csdord_str_cur = list(csdord_str_cur)
r = utils.makeSimpleRaw(np.abs(abscsd), sfreq=windowsz/skip, 
                        ch_names=csdord_str_cur,l=4, force_trunc_renum= 1 )
r.set_annotations(anns)
r.plot(scalings='auto', duration=times_pri[0][-1]);

In [ ]:
csdord_str_cur

In [ ]:
help(r.plot)

In [ ]:
csdord_strs

# From zero

In [ ]:
cross_types

In [ ]:
rl = roi_labels['all_raw']
rl.index('Precentral_R'), rl.index('Cerebellum_L'), rl.index('Rolandic_Oper_R'),\
    rl.index('Supp_Motor_Area_R'), rl.index('Postcentral_R')

In [ ]:
rl[4] # = 'Frontal_Sup_R' is not a motor region 

In [ ]:
1

In [ ]:
chnames_tfr0 = ['LFPR01', 'LFPR12', 
                'msrcR_0_2_c0', 'msrcR_0_2_c6', 'msrcR_0_4_c12', 
                'msrcL_0_60_c8', 'msrcL_0_60_c13']

In [ ]:
chnames_nicened0 = utils.nicenMEGsrc_chnames(chnames_tfr0, roi_labels, srcgrouping_names_sorted,
                        prefix='msrc_')


In [ ]:
res_couplings0 = ugf.selectIndPairs(chnames_nicened0, chnames_tfr0, cross_types, upper_diag=False,
                        LFP2LFP_only_self=LFP2LFP_only_self, cross_within_parcel=False)


In [ ]:
ind_distr0, ind_distr_parcels0, ind_pairs_parcelsLFP0, \
    parcel_couplings0, LFP2parcel_couplings0, LFP2LFP_couplings0 = res_couplings0


In [ ]:
cpl = LFP2LFP_couplings
for chn_tpl,chi_tpl_list in cpl.items():
    print(chn_tpl,chi_tpl_list)
    for chi_tpl in chi_tpl_list:
        print( chnames_tfr[chi_tpl[0]], chnames_tfr[chi_tpl[1]] )

In [ ]:
print(parcel_couplings0)

In [ ]:
cpl = parcel_couplings0
for chn_tpl,chi_tpl_list in cpl.items():
    print(chn_tpl,chi_tpl_list)
    for chi_tpl in chi_tpl_list:
        s0 = chnames_tfr0[chi_tpl[0]]
        s1 = chnames_tfr0[chi_tpl[1]]
        sr = ''
        if s0 == s1:
            sr = 'self_' + sr
        sr += '   {} {}'.format(s0,s1)
        print(sr)

In [ ]:
cpl = LFP2parcel_couplings0
for chn_tpl,chi_tpl_list in cpl.items():
    print(chn_tpl,chi_tpl_list)
    for chi_tpl in chi_tpl_list:
        s0 = chnames_tfr0[chi_tpl[0]]
        s1 = chnames_tfr0[chi_tpl[1]]
        sr = ''
        if s0 == s1:
            sr = 'self_' + sr
        sr += '   {} {}'.format(s0,s1)
        print(sr)

In [ ]:
freqs0 = [5, 7, 20]
n_cycles0 = freqs0
nfreq0 = len(freqs0)
ntimes0 = sfreq * 2

chi_test0 = 0; print(chnames_tfr0[chi_test0])
chi_test1 = 2; print(chnames_tfr0[chi_test1])
chi_test2 = 5; print(chnames_tfr0[chi_test2])

noise_sz = 1e-6
testfreq = freqs0[1]
ts = np.arange(ntimes0) / sfreq
dat0 = np.zeros((len(chnames_tfr0),ntimes0))
dat0[chi_test0] = 10 *np.sin(ts * testfreq*  2 * np.pi ) 
dat0[chi_test0] += noise_sz * np.random.uniform(low=-1,high=1,size=len(ts))

dat0[chi_test1] = 20 *np.sin(ts * testfreq*  2 * np.pi ) 
dat0[chi_test1] += noise_sz * np.random.uniform(low=-1,high=1,size=len(ts))

dat0[chi_test2] = 20 *np.sin(ts * testfreq*  2 * np.pi + 0.4*np.pi ) 
dat0[chi_test2] += noise_sz * np.random.uniform(low=-1,high=1,size=len(ts))

tfrres_0,wbd = utils.tfr(dat0, sfreq, freqs0, n_cycles0,
                        windowsz, decim = skip,
                        n_jobs=n_jobs)


In [ ]:
# LFP should couple to all, so I should get meaningul couplings for LFP to both sources
# also LFP to itself, these sources to itself and sources to each other

In [ ]:
plt.plot(ts,dat0[chi_test0])
plt.plot(ts,dat0[chi_test1])
plt.plot(ts,dat0[chi_test2])

In [ ]:
%matplotlib inline
ax1 = plt.subplot(2,1,1)
ax2 = plt.subplot(2,1,2)
chi_cur = chi_test0
for fi,f in enumerate(freqs0):
    ax1.plot(tfrres_0[chi_cur, fi].real, label='{}:{}'.format(fi,f ) )
    ax2.plot(tfrres_0[chi_cur, fi].imag, label='{}:{}'.format(fi,f ) )
ax1.legend(loc='upper right')
ax2.legend(loc='upper right')

In [ ]:
%matplotlib inline
ax1 = plt.subplot(2,1,1)
ax2 = plt.subplot(2,1,2)
chi_cur = chi_test2
for fi,f in enumerate(freqs0):
    ax1.plot(tfrres_0[chi_cur, fi].real, label='{}:{}'.format(fi,f ) )
    ax2.plot(tfrres_0[chi_cur, fi].imag, label='{}:{}'.format(fi,f ) )
ax1.legend(loc='upper right')
ax2.legend(loc='upper right')

In [ ]:
# chreate new chnames
parcels_present = []
pp2side = {}
for chn in chnames_tfr0:
    if chn.startswith('LFP'):
        continue
    side1, gi1, parcel_ind1, si1 = utils.parseMEGsrcChnameShort(chn)
    if parcel_ind1 in pp2side:
        assert pp2side[parcel_ind1] == side1, 'Side inconsistency within parcel!'
    pp2side[parcel_ind1] = side1
    parcels_present += [parcel_ind1]

pp = list(sorted(set(parcels_present)))
aa = ['msrc{}_{}_{}_c{}'.format(pp2side[p],newchn_grouping_ind,p,0) for p in pp]
lfpinds = utsne.selFeatsRegexInds(chnames_tfr0,'LFP.*')
newchns0 = aa + np.array(chnames_tfr)[lfpinds].tolist()


In [ ]:
newchns0

In [ ]:

#tfrres_cur0 = np.zeros( (len(chnames_tfr0), nfreq0,ntimes0))





In [ ]:
csd_cur0, csdord0 = ugf.tfr2csd(tfrres_0, sfreq, returnOrder=1,
                                ind_pairs=None,
                                parcel_couplings=parcel_couplings0,
                                LFP2LFP_couplings=LFP2LFP_couplings0,
                                LFP2parcel_couplings=LFP2parcel_couplings0,
                                oldchns=chnames_tfr0,
                                newchns=newchns0,
                                res_group_id=newchn_grouping_ind,
                                log=log_during_csd)


In [ ]:
chnames_tfr0

In [ ]:
print(csdord0)
for pair_ind in range(csdord0.shape[1]):
    i0,i1 = csdord0[:,pair_ind]
    print( newchns0[i0], newchns0[i1] )

In [ ]:
print(chnames_tfr0[chi_test0])
print(chnames_tfr0[chi_test1])
print(chnames_tfr0[chi_test2])
print(testfreq)

In [ ]:
#TODO: I should see bumps at freq = 7, should not I? Why are they at green curve?

In [ ]:
nr = csdord.shape[1]; nc=1
fig,axs = plt.subplots(nr,nc, figsize=(nc*10,nr*2))
axs = axs.reshape((nr,nc))
for i in range(nr):
    ax = axs[i,0]
    for fi,f in enumerate(freqs0):
        ax.plot( np.abs( csd_cur0[i,fi] ), label='{}: {}'.format(fi,f ) )
    ax.legend(loc='upper right')
    i0,i1 = csdord0[:,i]
    ax.set_title('{} vs {}'.format( newchns0[i0], newchns0[i1] ) )
plt.tight_layout()